In [17]:
import json
import numpy as np
import pandas as pd

data = './transaction-data-adhoc-analysis.json'
df = pd.read_json(data)

# separate transaction items
df["transaction_items"] = df["transaction_items"].str.split(";")
new_df = df.explode("transaction_items")

# separate month
new_df['transaction_month'] = pd.to_datetime(new_df['transaction_date']).dt.month
     
# amount of transaction items
amount = new_df['transaction_items'].str[-2]
new_df['items_amount'] = amount
new_df = new_df.astype({'items_amount':'int'})

#separate transaction items from brand
items = new_df['transaction_items'].str.split(pat = ',', expand = True)
new_df.insert(loc = 1, column = 'brand_name', value = items[0])
new_df.insert(loc = 2, column = 'item_name', value = items[1])

new_df = new_df[["name","item_name","transaction_value","items_amount","transaction_month"]]

new_df.head()
#here we were able to extract the month from the transaction date, each item, and the amount of items

,name,item_name,transaction_value,items_amount,transaction_month
0,Jennifer Campbell,Beef Chicharon,5196,4,1
1,Melissa Kim,Beef Chicharon,12056,3,1
1,Melissa Kim,Nutrional Milk,12056,4,1
1,Melissa Kim,Orange Beans,12056,1,1
2,Melissa Kim,Gummy Vitamins,5500,3,1


In [18]:
#Price per product
data = {'brand_name':  ['Candy City', 'Candy City', 'Exotic Extras', 'Exotic Extras', 'HealthyKid 3+', 'HealthyKid 3+', 'HealthyKid 3+'],
        'item_name': ['Gummy Worms','Orange Beans','Beef Chicharon','Kimchi and Seaweed','Gummy Vitamins','Nutritional Milk',',Yummy Vegetables'],
        'item_price': [150, 199, 1299, 799, 1500, 1990, 500],
        }

price_df = pd.DataFrame(data,index=['product_1','product_2','product_3','product_4','product_5', 'product_6', 'product_7'])

print (price_df)

              brand_name           item_name  item_price
product_1     Candy City         Gummy Worms         150
product_2     Candy City        Orange Beans         199
product_3  Exotic Extras      Beef Chicharon        1299
product_4  Exotic Extras  Kimchi and Seaweed         799
product_5  HealthyKid 3+      Gummy Vitamins        1500
product_6  HealthyKid 3+    Nutritional Milk        1990
product_7  HealthyKid 3+   ,Yummy Vegetables         500


In [19]:
df2 = new_df.merge(price_df, left_on=None, right_on=None)

df2

,name,item_name,transaction_value,items_amount,transaction_month,brand_name,item_price
0,Jennifer Campbell,Beef Chicharon,5196,4,1,Exotic Extras,1299
1,Melissa Kim,Beef Chicharon,12056,3,1,Exotic Extras,1299
2,Jennifer Figueroa,Beef Chicharon,10196,4,1,Exotic Extras,1299
3,Benjamin Wilson,Beef Chicharon,3596,2,1,Exotic Extras,1299
4,Melissa Edwards,Beef Chicharon,2996,2,1,Exotic Extras,1299
...,...,...,...,...,...,...,...
118750,Bethany Peters,Kimchi and Seaweed,12656,4,6,Exotic Extras,799
118751,Bethany Peters,Kimchi and Seaweed,1399,1,6,Exotic Extras,799
118752,Tammy Byrd,Kimchi and Seaweed,4779,1,6,Exotic Extras,799
118753,Tammy Byrd,Kimchi and Seaweed,3588,2,6,Exotic Extras,799


In [23]:
df2['total'] = df2['items_amount'] * df2['item_price']

df2

,name,item_name,transaction_value,items_amount,transaction_month,brand_name,item_price,total
0,Jennifer Campbell,Beef Chicharon,5196,4,1,Exotic Extras,1299,5196
1,Melissa Kim,Beef Chicharon,12056,3,1,Exotic Extras,1299,3897
2,Jennifer Figueroa,Beef Chicharon,10196,4,1,Exotic Extras,1299,5196
3,Benjamin Wilson,Beef Chicharon,3596,2,1,Exotic Extras,1299,2598
4,Melissa Edwards,Beef Chicharon,2996,2,1,Exotic Extras,1299,2598
...,...,...,...,...,...,...,...,...
118750,Bethany Peters,Kimchi and Seaweed,12656,4,6,Exotic Extras,799,3196
118751,Bethany Peters,Kimchi and Seaweed,1399,1,6,Exotic Extras,799,799
118752,Tammy Byrd,Kimchi and Seaweed,4779,1,6,Exotic Extras,799,799
118753,Tammy Byrd,Kimchi and Seaweed,3588,2,6,Exotic Extras,799,1598


In [24]:
# breakdown of the count of each item sold per month, and they also want to have a breakdown of the total sale value per item per month.

monthly_breakdown = pd.pivot_table(data=df2,
                       index=['item_name','transaction_month'],
                       values=['items_amount','total'],
                       aggfunc='sum')
monthly_breakdown

items_amount     total
item_name          transaction_month                        
Beef Chicharon     1                          9665  12554835
                   2                         10001  12991299
                   3                          9816  12750984
                   4                          9890  12847110
                   5                         10028  13026372
                   6                          9902  12862698
Gummy Vitamins     1                          9681  14521500
                   2                          9980  14970000
                   3                         10145  15217500
                   4                          9842  14763000
                   5                          9948  14922000
                   6                          9980  14970000
Gummy Worms        1                          9559   1433850
                   2                          9996   1499400
                   3                          9986   1497900
                   4                         10043   1506450
                   5                          9801   1470150
                   6                          9934   1490100
Kimchi and Seaweed 1                          9676   7731124
                   2                          9949   7949251
                   3                          9967   7963633
                   4                          9921   7926879
                   5                          9773   7808627
                   6                         10104   8073096
Orange Beans       1                          9774   1945026
                   2                         10037   1997363
                   3                          9611   1912589
                   4                          9914   1972886
                   5                          9964   1982836
                   6                         10106   2011094

In [26]:
customer_data = pd.pivot_table(data=df2,
                       index=['transaction_month'],
                       columns=['name'],
                       values=['items_amount'],
                       aggfunc='sum')
customer_data

items_amount                                        \
name              Aaron Beasley Aaron Brewer Aaron Brown Aaron Coffey   
transaction_month                                                       
1                           NaN          3.0         NaN          NaN   
2                           NaN          3.0         NaN          4.0   
3                           NaN          NaN        16.0          8.0   
4                           8.0          5.0        14.0         12.0   
5                           8.0          NaN         3.0          6.0   
6                           1.0         10.0         1.0          NaN   

                                                                        \
name              Aaron Davis Aaron Escobar Aaron Flores Aaron Flowers   
transaction_month                                                        
1                         2.0           NaN          6.0          10.0   
2                         NaN           3.0         15.0           1.0   
3                         NaN           NaN          5.0           NaN   
4                        15.0           NaN          8.0           2.0   
5                        10.0           4.0          NaN           NaN   
6                         5.0           NaN          1.0           5.0   

                                              ...                  \
name              Aaron Garrett Aaron Holder  ... Zachary Richard   
transaction_month                             ...                   
1                           7.0          NaN  ...            17.0   
2                           7.0          7.0  ...            11.0   
3                           4.0          3.0  ...             6.0   
4                           NaN          3.0  ...            20.0   
5                           NaN          4.0  ...             7.0   
6                           4.0          5.0  ...             7.0   

                                                                              \
name              Zachary Salinas Zachary Scott Zachary Smith Zachary Torres   
transaction_month                                                              
1                             NaN          10.0           5.0            NaN   
2                             NaN          10.0           6.0            NaN   
3                             NaN           2.0          13.0           12.0   
4                             3.0          10.0          28.0            2.0   
5                             4.0           7.0          19.0            5.0   
6                             NaN           5.0          38.0            5.0   

                                                                   \
name              Zachary Valentine Zachary Ware Zachary Williams   
transaction_month                                                   
1                               5.0          6.0             20.0   
2                               6.0          3.0             14.0   
3                               2.0         11.0             15.0   
4                               2.0          5.0             11.0   
5                              10.0          5.0              NaN   
6                              17.0          7.0              NaN   

                                               
name              Zachary Wilson Zachary York  
transaction_month                              
1                            NaN          3.0  
2                            NaN         13.0  
3                            NaN          4.0  
4                            4.0         12.0  
5                            4.0         20.0  
6                            3.0          5.0  

[6 rows x 8486 columns]

In [34]:
#change nan to 0
customer_data.fillna(0)

# Repeaters - the number of customers from the current month who also purchased in the previous month. This metric is 0 for the earliest month in the transactional data.

# Inactive - the number of customers in the total set of transactions up to and including the current month who have purchase history but do not have a purchase for the current month. This metric is 0 for the earliest month in the transactional data.
inactive = {"1":0
            "2":

items_amount                                        \
name              Aaron Beasley Aaron Brewer Aaron Brown Aaron Coffey   
transaction_month                                                       
1                           0.0          3.0         0.0          0.0   
2                           0.0          3.0         0.0          4.0   
3                           0.0          0.0        16.0          8.0   
4                           8.0          5.0        14.0         12.0   
5                           8.0          0.0         3.0          6.0   
6                           1.0         10.0         1.0          0.0   

                                                                        \
name              Aaron Davis Aaron Escobar Aaron Flores Aaron Flowers   
transaction_month                                                        
1                         2.0           0.0          6.0          10.0   
2                         0.0           3.0         15.0           1.0   
3                         0.0           0.0          5.0           0.0   
4                        15.0           0.0          8.0           2.0   
5                        10.0           4.0          0.0           0.0   
6                         5.0           0.0          1.0           5.0   

                                              ...                  \
name              Aaron Garrett Aaron Holder  ... Zachary Richard   
transaction_month                             ...                   
1                           7.0          0.0  ...            17.0   
2                           7.0          7.0  ...            11.0   
3                           4.0          3.0  ...             6.0   
4                           0.0          3.0  ...            20.0   
5                           0.0          4.0  ...             7.0   
6                           4.0          5.0  ...             7.0   

                                                                              \
name              Zachary Salinas Zachary Scott Zachary Smith Zachary Torres   
transaction_month                                                              
1                             0.0          10.0           5.0            0.0   
2                             0.0          10.0           6.0            0.0   
3                             0.0           2.0          13.0           12.0   
4                             3.0          10.0          28.0            2.0   
5                             4.0           7.0          19.0            5.0   
6                             0.0           5.0          38.0            5.0   

                                                                   \
name              Zachary Valentine Zachary Ware Zachary Williams   
transaction_month                                                   
1                               5.0          6.0             20.0   
2                               6.0          3.0             14.0   
3                               2.0         11.0             15.0   
4                               2.0          5.0             11.0   
5                              10.0          5.0              0.0   
6                              17.0          7.0              0.0   

                                               
name              Zachary Wilson Zachary York  
transaction_month                              
1                            0.0          3.0  
2                            0.0         13.0  
3                            0.0          4.0  
4                            4.0         12.0  
5                            4.0         20.0  
6                            3.0          5.0  

[6 rows x 8486 columns]

In [37]:
# replace nan values
customer_data.replace(np.nan,"",regex=True,inplace=True)

# function to change table to true and false
def did_order(x):
    if x == "":
        return False
    else:
        return True

# apply function to all values
customer_data = customer_data.applymap(category)

# make column names referenceable strings
customer_data.columns = ['1','2','3','4','5','6']

# make copies of the table to check for each type
pivot_table_users_repeaters = customer_data.copy()
pivot_table_users_inactive = customer_data.copy()
pivot_table_users_engaged = customer_data.copy()

## fixing repeaters data
pivot_table_users_repeaters['2'] = np.where((pivot_table_users['1'] == True) & 
                                          (pivot_table_users['2'] == True),
                                          1,np.nan)

pivot_table_users_repeaters['3'] = np.where((pivot_table_users['2'] == True) & 
                                          (pivot_table_users['3'] == True),
                                          1,np.nan)

pivot_table_users_repeaters['4'] = np.where((pivot_table_users['3'] == True) & 
                                          (pivot_table_users['4'] == True),
                                          1,np.nan)

pivot_table_users_repeaters['5'] = np.where((pivot_table_users['4'] == True) & 
                                          (pivot_table_users['5'] == True),
                                          1,np.nan)

pivot_table_users_repeaters['6'] = np.where((pivot_table_users['5'] == True) & 
                                          (pivot_table_users['6'] == True),
                                          1,np.nan)

pivot_table_users_repeaters['1'] = np.nan

ValueError: Length mismatch: Expected axis has 8486 elements, new values have 6 elements

In [36]:
x = customer_data
Repeater= {"1": 0, 
           "2": list(x.loc[(x[1]>= 1)& (x[2]>= 1)].count()-1)[0], 
           "3": list(x.loc[(x[2]>= 1)& (x[3]>= 1)].count()-1)[0],
           "4": list(x.loc[(x[3]>= 1)& (x[4]>= 1)].count()-1)[0], 
           "5": list(x.loc[(x[4]>= 1)& (x[5]>= 1)].count()-1)[0], 
           "6": list(x.loc[(x[5]>= 1)& (x[6]>= 1)].count()-1)[0] 
          }

Inactive = {"1": 0, 
           "2": list(x.loc[(x[1]>= 1)& (x[2]== 0)].count())[0],
           "3": list(x.loc[(x[1]>= 0)& (x[2]>= 0)& (x[3]== 0)].count())[0]-list(x.loc[(x[1]== 0)& (x[2]== 0)& (x[3]== 0)].count())[0],
           "4": list(x.loc[(x[1]>= 0)& (x[2]>= 0)& (x[3]>= 0)& (x[4]== 0)].count())[0]-list(x.loc[(x[1]== 0)& (x[2]== 0)& (x[3]== 0)& (x[4]== 0)].count())[0], 
           "5": list(x.loc[(x[1]>= 0)& (x[2]>= 0)& (x[3]>= 0)& (x[4]>= 0)& (x[5]== 0)].count())[0]-list(x.loc[(x[1]== 0)& (x[2]== 0)& (x[3]== 0)& (x[4]== 0)& (x[5]== 0)].count())[0], 
           "6": list(x.loc[(x[1]>= 0)& (x[2]>= 0)& (x[3]>= 0)& (x[4]>= 0)& (x[5]>= 0)& (x[6]== 0)].count())[0]-list(x.loc[(x[1]== 0)& (x[2]== 0)& (x[3]== 0)& (x[4]== 0)& (x[5]== 0)& (x[6]== 0)].count())[0] 
          }

Engaged = {"1": list(x.loc[(x[1]>= 1)].count()-1)[0], 
           "2": list(x.loc[(x[1]>= 1)& (x[2]>= 1)].count()-1)[0],
           "3": list(x.loc[(x[1]>= 1)& (x[2]>= 1)& (x[3]>= 1)].count()-1)[0],
           "4": list(x.loc[(x[1]>= 1)& (x[2]>= 1)& (x[3]>= 1)& (x[4]>= 1)].count()-1)[0], 
           "5": list(x.loc[(x[1]>= 1)& (x[2]>= 1)& (x[3]>= 1)& (x[4]>= 1)& (x[5]>= 1)].count()-1)[0], 
           "6": list(x.loc[(x[1]>= 1)& (x[2]>= 1)& (x[3]>= 1)& (x[4]>= 1)& (x[5]>= 1)& (x[6]>= 1)].count()-1)[0] 
          }

KeyError: 1